# Bird Object Detection - Model Deployment

Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This lets us make predictions (or inferences) from the model. Note that we don’t have to host using the same type of instance that we used to train. Training is a prolonged and compute heavy job with different compute and memory requirements that hosting typically does not. In our case we chose the ml.p3.2xlarge instance to train, but we choose to host the model on the less expensive cpu instance, ml.m5.xlarge. The endpoint deployment takes several minutes, and can be accomplished with a single line of code calling the deploy method.

Please find your training job name from the left-hand SageMaker Studio tab 'Components and Registries' and select 'Experiments and Trials'. Click on 'Unassigned trial components' and find your latest training job run. Click on it and expand details, where you will find the 'Job name' attribute.

!![TrainingJobName](./images/JobName.png)

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

sess = sagemaker.Session()
# this will create a 'default' sagemaker bucket if it doesn't exist (sagemaker-region-accountid)
bucket = sess.default_bucket()

role = get_execution_role()

TRAINING_JOB_NAME = "<insert_training_job_name_here>"

od_estimator = Estimator.attach(TRAINING_JOB_NAME)

In [ ]:
%%time

object_detector = od_estimator.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

Let's download images from the types of birds we trained our model to recognize. These images have not been seen by the algorithm during training and are not part of the original dataset.

In [ ]:
!mkdir -p unseen

!wget -q -O unseen/multi-goldfinch-1.jpg https://t3.ftcdn.net/jpg/01/44/64/36/500_F_144643697_GJRUBtGc55KYSMpyg1Kucb9yJzvMQooW.jpg
!wget -q -O unseen/northern-flicker-1.jpg https://upload.wikimedia.org/wikipedia/commons/5/5c/Northern_Flicker_%28Red-shafted%29.jpg
!wget -q -O unseen/northern-cardinal-1.jpg https://cdn.pixabay.com/photo/2013/03/19/04/42/bird-94957_960_720.jpg
!wget -q -O unseen/blue-jay-1.jpg https://cdn12.picryl.com/photo/2016/12/31/blue-jay-bird-feather-animals-b8ee04-1024.jpg
!wget -q -O unseen/hummingbird-1.jpg http://res.freestockphotos.biz/pictures/17/17875-hummingbird-close-up-pv.jpg

And from our classes file, let's get the Class IDs for the specific bird classes we filtered out from our birds dataset.

In [ ]:
import pandas as pd

BASE_DIR = "CUB_200_2011/"
IMAGES_DIR = BASE_DIR + "images/"

CLASSES_FILE = BASE_DIR + "classes.txt"
CLASS_COLS = ["class_number", "class_id"]

classes_df = pd.read_csv(CLASSES_FILE, sep=" ", names=CLASS_COLS, header=None)

CLASSES = [17, 36, 47, 68, 73]

criteria = classes_df["class_number"].isin(CLASSES)
classes_df = classes_df[criteria]
OBJECT_CATEGORIES = classes_df["class_id"].values.tolist()

print(OBJECT_CATEGORIES)

We now run inference for these images against our endpoint, we can see the details on the JSON response on the Object Detection SageMaker algorithm documentation pages here: https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection-in-formats.html . You can see that the predictions will include **one or more** bounding box coordinates along with predicted label and confidence score.

We threshold the predictions to only visualise those above a certain confidence score, that can be passed in the 'thresh' argument of our util **visualize_detection** helper function.

In [ ]:
import os
from sagemaker.serializers import IdentitySerializer
import utils
import json

img_serializer = IdentitySerializer(content_type='image/jpeg')
object_detector.serializer = img_serializer

for filename in os.listdir('./unseen'):
    if filename.endswith(".jpg"):
        filepath = os.path.join('./unseen', filename)
        with open(filepath, "rb") as image:
            f = image.read()
            img_data = bytearray(f)
        print("Predicting class for {}".format(filename))
        dets = json.loads(object_detector.predict(img_data))
        utils.visualize_detection(filepath, dets["prediction"], OBJECT_CATEGORIES, thresh=0.4)

In [ ]:

### Bonus!

Play around with the threshold to see the types of bounding boxes that are returned at different confidence levels.